# Previsão de abandono de disciplinas da Computação - UFABC
### by Jairo Freitas e colaboradores

Os dados foram preparados na sua maior parte por Bruno A. Nesta etapa ajustaremos modelos com vistas a predição do abandono de disciplinas entre os alunos da computação

## DataPrep para modelagem

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('dataset.csv', sep='|')

In [12]:
df.columns

Index(['Unnamed: 0', 'ID DO ALUNO', 'ANO', 'QUADRIMESTRE', 'DISCIPLINA',
       'COD_DISCIPLINA', 'CATEGORIA', 'ANO_QUAD', 'QUAD_CURSADO',
       'ANOS_NA_UNIVER', 'SIGLA', 'T', 'P', 'I', 'CRED', 'CONCEITO',
       'CONCEITO_CRED', 'PORC_APROV', 'PORC_REPROV', 'PORC_ABANDO',
       'CRED_POR_QUAD', 'DIFERENCA_A_MEDIA', 'Z_SCORE', 'CR_ACUMULADO',
       'CR_QUAD', 'CR_ATE_QUAD_ANTERIOR', 'CR_POR_ANO', 'A', 'B', 'C', 'D',
       'F', 'O', 'APROVADO', 'REPROVADO', 'ABANDONO'],
      dtype='object')

In [15]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [160]:
df.head()

,ID DO ALUNO,ANO,QUADRIMESTRE,DISCIPLINA,COD_DISCIPLINA,CATEGORIA,ANO_QUAD,QUAD_CURSADO,ANOS_NA_UNIVER,SIGLA,...,CR_POR_ANO,A,B,C,D,F,O,APROVADO,REPROVADO,ABANDONO
0,1.0,2011.0,2º Quadrimestre,bases computacionais da ciencia,BIM0005-13,Obrigatória,2011.02º Quadrimestre,1,0,BIS0005-15,...,1.7,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1.0,2011.0,2º Quadrimestre,bases matematicas,BIN0003-13,Obrigatória,2011.02º Quadrimestre,1,0,BIS0003-15,...,1.7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,1.0,2011.0,2º Quadrimestre,iniciacao a pesquisa cientifica i,ESZX129-13,Opção Limitada,2011.02º Quadrimestre,1,0,BC2001,...,1.7,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,2011.0,2º Quadrimestre,origem da vida e diversidade dos seres vivos,BIL0304-13,Obrigatória,2011.02º Quadrimestre,1,0,BC0304,...,1.7,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,2011.0,3º Quadrimestre,geometria analitica,BCN0404-13,Obrigatória,2011.03º Quadrimestre,2,0,BCN0404-15,...,1.7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [129]:
df_small = df[['ID DO ALUNO', 'ANO', 'QUADRIMESTRE', 'DISCIPLINA',
       'COD_DISCIPLINA', 'CATEGORIA', 'QUAD_CURSADO',
       'ANOS_NA_UNIVER', 'SIGLA', 'T', 'P', 'I', 'CRED', 'CONCEITO',
       'PORC_APROV', 'PORC_REPROV', 'PORC_ABANDO', 'CRED_POR_QUAD', 
        'DIFERENCA_A_MEDIA', 'Z_SCORE', 'CR_ATE_QUAD_ANTERIOR','APROVADO', 
        'REPROVADO', 'ABANDONO']]

In [130]:
#Gerando variáveis booleanas para categorias de disciplinas
df_small = pd.get_dummies(df_small,columns=['CATEGORIA'])

### Seleção e renomeamento de colunas

* Colunas iniciadas com **A_** contêm informações sobre o aluno
* Colunas iniciadas com **D_** contêm informações sobre a disciplina

In [131]:
df_small.columns

Index(['ID DO ALUNO', 'ANO', 'QUADRIMESTRE', 'DISCIPLINA', 'COD_DISCIPLINA',
       'QUAD_CURSADO', 'ANOS_NA_UNIVER', 'SIGLA', 'T', 'P', 'I', 'CRED',
       'CONCEITO', 'PORC_APROV', 'PORC_REPROV', 'PORC_ABANDO', 'CRED_POR_QUAD',
       'DIFERENCA_A_MEDIA', 'Z_SCORE', 'CR_ATE_QUAD_ANTERIOR', 'APROVADO',
       'REPROVADO', 'ABANDONO', 'CATEGORIA_(vazio)', 'CATEGORIA_Erro!',
       'CATEGORIA_Livre Escolha', 'CATEGORIA_Obrigatória',
       'CATEGORIA_Opção Limitada'],
      dtype='object')

In [132]:
df_small.drop(['CATEGORIA_(vazio)', 'CATEGORIA_Erro!'],axis=1,inplace=True)

In [133]:
df_small.columns

Index(['ID DO ALUNO', 'ANO', 'QUADRIMESTRE', 'DISCIPLINA', 'COD_DISCIPLINA',
       'QUAD_CURSADO', 'ANOS_NA_UNIVER', 'SIGLA', 'T', 'P', 'I', 'CRED',
       'CONCEITO', 'PORC_APROV', 'PORC_REPROV', 'PORC_ABANDO', 'CRED_POR_QUAD',
       'DIFERENCA_A_MEDIA', 'Z_SCORE', 'CR_ATE_QUAD_ANTERIOR', 'APROVADO',
       'REPROVADO', 'ABANDONO', 'CATEGORIA_Livre Escolha',
       'CATEGORIA_Obrigatória', 'CATEGORIA_Opção Limitada'],
      dtype='object')

In [134]:
df_small.columns = ['A_ID DO ALUNO', 'A_ANO', 'A_QUADRIMESTRE', 'D_DISCIPLINA',
       'D_COD_DISCIPLINA','A_QUAD_CURSADO', 'A_ANOS_NA_UNIVER', 'D_SIGLA', 
        'D_T', 'D_P', 'D_I', 'D_CRED', 'A_CONCEITO', 'D_PORC_APROV', 
        'D_PORC_REPROV', 'D_PORC_ABANDO', 'A_CRED_POR_QUAD', 'A_DIFF_CRED_POR_QUAD_A_MEDIA', 
        'A_Z_SCORE_CRED_POR_QUAD', 'A_CR_ATE_QUAD_ANTERIOR','A_APROVADO', 
        'A_REPROVADO', 'A_ABANDONO', 'D_LIVRE', 'D_OBRIGATORIA',
       'D_LIMITADA']
df_small = df_small.dropna()

In [164]:
X = df_small[['A_QUAD_CURSADO', 'A_ANOS_NA_UNIVER',
        'D_T', 'D_P', 'D_I', 'D_CRED', 'D_PORC_APROV', 
        'D_PORC_REPROV', 'D_PORC_ABANDO', 'A_Z_SCORE_CRED_POR_QUAD', 
        'A_CR_ATE_QUAD_ANTERIOR', 'D_LIVRE', 'D_OBRIGATORIA',
       'D_LIMITADA']]
y = df_small[['A_ABANDONO']]

### Dimensionamento das colunas

In [165]:
X.head(17)

,A_QUAD_CURSADO,A_ANOS_NA_UNIVER,D_T,D_P,D_I,D_CRED,D_PORC_APROV,D_PORC_REPROV,D_PORC_ABANDO,A_Z_SCORE_CRED_POR_QUAD,A_CR_ATE_QUAD_ANTERIOR,D_LIVRE,D_OBRIGATORIA,D_LIMITADA
0,1,0,0,2,2,2,0.937107,0.044025,0.018868,0.283190,4.000000,0,1,0
1,1,0,4,0,5,4,0.758475,0.241525,0.000000,0.283190,4.000000,0,1,0
2,1,0,2,0,3,2,1.000000,0.000000,0.000000,0.283190,4.000000,0,0,1
3,1,0,3,0,4,3,0.942149,0.057851,0.000000,0.283190,4.000000,0,1,0
4,2,0,3,0,6,3,0.628571,0.331429,0.040000,-0.323645,1.727273,0,1,0
5,2,0,3,0,4,3,0.938272,0.043210,0.018519,-0.323645,1.727273,0,1,0
6,2,0,3,0,4,3,0.968586,0.026178,0.005236,-0.323645,1.727273,0,1,0
7,3,1,4,0,5,4,0.758475,0.241525,0.000000,2.103695,1.700000,0,1,0
8,3,1,3,0,4,3,0.918269,0.048077,0.033654,2.103695,1.700000,0,1,0
9,3,1,3,2,5,5,0.850932,0.136646,0.012422,2.103695,1.700000,0,1,0


### Separação dos dados de treinamento e teste
OBS: Usamos estratificação

In [166]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42, stratify=y)
y_train = y_train['A_ABANDONO'].values.tolist()
y_test = y_test['A_ABANDONO'].values.tolist()

/home/jairo/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [167]:
print("Train: ",X_train.shape[0],"amostras com ", sum(y_train)/X_train.shape[0]," abandonos")
print("Test: ",X_test.shape[0],"amostras com ", sum(y_test)/X_test.shape[0]," abandonos")

Train:  9185 amostras com  0.03799673380511704  abandonos
Test:  2297 amostras com  0.037875489769264255  abandonos


## Modelagem

In [168]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score

def eval_model(model_name, y, y_pred):
    print("[",model_name,"]")
    print("Accuracy: ",accuracy_score(y, y_pred))
    print("Precision: ",precision_score(y, y_pred))
    print("Recall: ",recall_score(y, y_pred))  

In [169]:
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(random_state=42)
dummy_train_score = cross_val_score(dummy, X_train, y_train, cv=10).mean()
dummy.fit(X_train, y_train)

y_pred_dummy = dummy.predict(X_test)

eval_model("Dummy", y_test, y_pred_dummy)

[ Dummy ]
Accuracy:  0.9233783195472355
Precision:  0.041237113402061855
Recall:  0.04597701149425287


In [172]:
from sklearn.linear_model import LogisticRegressionCV 

lrn = LogisticRegressionCV(cv=10, Cs=10 ,penalty='l2', 
                           scoring='recall', solver='liblinear', random_state=42).fit(X_train, y_train)

In [173]:
y_pred_lrn = lrn.predict(X_test)

eval_model("Logistic Regression", y_test, y_pred_lrn)

[ Logistic Regression ]
Accuracy:  0.9638659120592077
Precision:  0.6428571428571429
Recall:  0.10344827586206896
